In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras import Model
from tensorflow.keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
#from matplotlib import pyplot
from tensorflow.keras import regularizers
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
X_train = load('trainX1.npy')
X_test = load('testX1.npy')
y_train = load('trainY1.npy')
y_test = load('testY1.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(33104, 128, 6) (33104, 118) (3740, 128, 6) (3740, 118)


In [3]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

# 1-LSTM

In [4]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [5]:
model1 = Sequential()
#forward_layer =LSTM(500,input_shape=(n_timesteps,n_features),return_sequences=True)
#backward_layer =LSTM(500,activation='relu',return_sequences=True,go_backwards=True)
#model1.add(Bidirectional(LSTM(500, input_shape=(n_timesteps,n_features))))
#model1.add(Bidirectional(LSTM(500, input_shape=(n_timesteps,n_features))))
model1.add(Bidirectional(LSTM(256, return_sequences=False), input_shape=(n_timesteps,n_features)))
#model1.add(Bidirectional(LSTM(500)))
model1.add(Dense(256, activation='relu'))
model1.add(Dropout(0.1))
model1.add(Dense(128, activation='relu'))
#model1.add(Dropout(0.1))
#model1.add(Dense(100, activation='relu'))
model1.add(Dense(n_outputs, activation='softmax'))
model1.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model1.summary()
callbacks = [tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 512)               538624    
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 118)               15222     
Total params: 718,070
Trainable params: 718,070
Non-trainable params: 0
_________________________________________________________________


In [7]:
history = model1.fit(X_training, y_training, epochs=200, verbose=True, validation_data=(X_validation, y_validation), batch_size=128,callbacks=callbacks)

Epoch 1/200
207/207 [==============================] - 551s 3s/step - loss: 0.0029 - accuracy: 0.7547 - val_loss: 0.0019 - val_accuracy: 0.8395
Epoch 2/200
207/207 [==============================] - 1955s 9s/step - loss: 0.0015 - accuracy: 0.8741 - val_loss: 0.0012 - val_accuracy: 0.9021
Epoch 3/200
207/207 [==============================] - 931s 4s/step - loss: 0.0011 - accuracy: 0.9104 - val_loss: 0.0011 - val_accuracy: 0.9056
Epoch 4/200
207/207 [==============================] - 598s 3s/step - loss: 8.8597e-04 - accuracy: 0.9259 - val_loss: 9.2869e-04 - val_accuracy: 0.9222
Epoch 5/200
207/207 [==============================] - 459s 2s/step - loss: 7.1712e-04 - accuracy: 0.9400 - val_loss: 7.2160e-04 - val_accuracy: 0.9393
Epoch 6/200
207/207 [==============================] - 456s 2s/step - loss: 5.7612e-04 - accuracy: 0.9524 - val_loss: 6.2616e-04 - val_accuracy: 0.9476
Epoch 7/200
207/207 [==============================] - 467s 2s/step - loss: 5.2704e-04 - accuracy: 0.9565 - val

In [8]:
loss, accuracy = model1.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model1.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9754
Testing Accuracy: 0.9056
